In [ ]:
def check_gpu():
    import tensorflow as tf
    print("tf.test.is_built_with_cuda()")
    print(tf.test.is_built_with_cuda())
    
    print()
    print("tf.config.list_physical_devices('GPU')")
    print(tf.config.list_physical_devices('GPU'))

    print()
    print("tf.config.experimental.list_physical_devices('GPU')")
    print(tf.config.experimental.list_physical_devices('GPU'))

#check_gpu()

In [ ]:
import pandas as pd

def generateData():    
    chess_evals = pd.read_csv("chessData.csv")
    random_evals = pd.read_csv("random_evals.csv")
    tactic_evals = pd.read_csv("tactic_evals.csv")
    df = pd.concat([chess_evals, random_evals, tactic_evals], join="inner")
    df = df.drop_duplicates(subset=df.columns[0]) # remove repeated rows
    df = df.sample(frac=1, random_state=3)  # frac=1 shuffles all rows
    
    trainFile = open("train.csv", "w")
    testFile = open("test.csv", "w")
    trainFile.write("FEN,Evaluation\n")    
    testFile.write("FEN,Evaluation\n") 
        
    def clamp(x):
        if x <= -1000:
            return -1000
        if x >= 1000:
            return 1000
        return x
    
    from stockfish import Stockfish
    parameters = {"Threads": 4, "Hash": 1024}
    stockfish = Stockfish(path="/users/ricar/desktop/stockfish/stockfish-windows-x86-64-avx2.exe", depth=9, parameters=parameters)
    
    i = 0
    for fen in df["FEN"].values:
        stockfish.set_fen_position(fen)
        eval = stockfish.get_evaluation()
        if eval["type"] != "cp":
            continue
        value = clamp(int(eval["value"]))
        i += 1
        if i % 5 == 0:
            testFile.write(fen + "," + str(value) + "\n")
        else:
            trainFile.write(fen + "," + str(value) + "\n")
        if i % 100 == 0:
            print(i, "evaluated" end="\r")
        
generateData()

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import pandas as pd

def countLines(filename):
    with open(filename, "r") as file:
        line_count = sum(1 for line in file)
    return line_count

numTrainExamples = countLines("train.csv") - 1
numTestExamples = countLines("test.csv") - 1
ratio = numTestExamples / (numTrainExamples + numTestExamples)
print("numTrainExamples", numTrainExamples)
print("numTestExamples", numTestExamples)
print("test " + str(round(ratio,2) * 100) + "%")

def loadData(file, idx, batch_size):
    df = pd.read_csv(file, skiprows=idx*batch_size, nrows=batch_size)
    x = [fen_to_12_8x8(fen) for fen in df[df.columns[0]].tolist()]
    y = df[df.columns[1]].tolist()
    return (np.array(x), np.array(y))

def batchGenerator(file, batch_size, steps):
    idx=1
    while True: 
        yield loadData(file, idx-1, batch_size) # Yields data
        if idx<steps:
            idx+=1
        else:
            idx=1

In [ ]:
import numpy as np
import chess

def fen_to_12_8x8(myFen, flip=True):
    whiteToMove = " w " in myFen
    if not flip or whiteToMove:
        board = chess.Board(myFen)
    elif flip and not whiteToMove:
        splitted = myFen.split(" ")
        splitted[0] = splitted[0].swapcase() # Flip piece colors
        # Now we have to invert the rows
        rows = splitted[0].split("/")
        leftPartReversed = ""
        for i in range(len(rows)-1, -1, -1): # start stop step
            leftPartReversed += rows[i]
            if i != 0:
                leftPartReversed += "/"
        splitted[0] = leftPartReversed
        fen = " ".join(splitted)
        board = chess.Board(fen)
    
    pieces = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    array = np.empty((12,8,8), dtype=int)
    i = 0 
    for color in [chess.WHITE, chess.BLACK]:
        for piece in pieces:
            bitboard = np.asarray(board.pieces(piece, color).tolist()).astype(int)
            bitboard = bitboard.reshape(8,8)
            array[i] = bitboard
            i += 1
            
    return array

# rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
#fen_to_12_8x8("7k/7p/6p1/4pp2/4P3/8/3Q1PPP/7K w - - 0 1")
#print()
#fen_to_12_8x8("7k/7p/6p1/4pp2/4P3/8/3Q1PPP/7K b - - 0 1")

In [ ]:
import pickle

def train():
    model = Sequential()
    model.add(Flatten(name="flatten", input_shape=(12, 8, 8)))  # Flatten the 3D input to a 1D array
    model.add(Dense(name="dense32", units=32, activation='relu'))
    model.add(Dense(name="dense1", units=1, activation='linear'))
    model.compile(loss="mae", optimizer=Adam(learning_rate=0.001)) # default lr 0.001
    model.summary()

    batch_size = 16
    epochs = 30
    steps_per_epoch = np.ceil(numTrainExamples/batch_size)
    validation_steps = np.ceil(numTestExamples/batch_size)
    my_training_batch_generator = batchGenerator('train.csv', batch_size, steps_per_epoch)
    my_validation_batch_generator = batchGenerator('test.csv', batch_size, validation_steps)

    history = model.fit(my_training_batch_generator,
                        epochs=epochs,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=my_validation_batch_generator,
                        validation_steps=validation_steps)
    
    model.save("model.h5")
    with open("history", 'wb') as hist:
        pickle.dump(history.history, hist) 
    
train()

In [ ]:
from tensorflow.keras.models import load_model
    
model = load_model("model.h5")
with open("history", "rb") as hist_file:
    history = pickle.load(hist_file)
model.summary()

In [ ]:
print("Evaluating on test set...")
loss = model.evaluate(X_test, y_test)

In [ ]:
def plot():
    import matplotlib.pyplot as plt
    global history
    if not isinstance(history, dict):
        history = history.history
    # Plot training and validation loss over epochs
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss (RMSE)')
    plt.legend()
    plt.show()
    
plot()

In [ ]:
def printWeightsForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        weights = layer.get_weights()
        if len(weights) > 0:
            rows, cols = weights[0].shape
            if cols > 1:
                print(f"double {layer.name}[{rows}][{cols}] = {{", end="")
                for row in weights[0]:
                    print("{", ",".join(str(val) for val in row), "},", end="")
            else:
                print(f"double {layer.name}[{rows}] = {{", end="")
                for row in weights[0]:
                    print(",".join(str(val) for val in row), ",", end="")

            print("};")
            print()
        print()
        
printWeightsForCpp(model)

In [ ]:
def printBiasesForCpp(model):
    for layer in model.layers:
        print("------ LAYER", layer.name, "------")
        biases = layer.get_weights()[1]
        print(f"double biases[{len(biases)}] = {{", end="")
        print(",".join(str(val) for val in biases), ",")
        print()
        
printBiasesForCpp(model)

In [ ]:
input = np.array([i % 4 for i in range(769)])
input = input.reshape(1, 769)

predictions = model.predict(input)
print("Prediction:", predictions[0][0])